In [ ]:
import sys
from pathlib import Path
# Make sure 'src' and project root are on sys.path regardless of the notebook working directory
cwd = Path.cwd()
if (cwd / 'src').exists():
    ROOT = cwd
else:
    ROOT = cwd.parent
print('Notebook root is', ROOT)
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
if str(ROOT / 'src') not in sys.path:
    sys.path.insert(0, str(ROOT / 'src'))
import importlib
importlib.invalidate_caches()

import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import differential_evolution, least_squares
from src.models.leloup_goldbeter import f, default_initial_conditions, LGParams

In [ ]:
# Define a loss function: RMSE across selected genes/time points.
from src.estimation.estimation import residuals_for_theta, theta_to_params, fit_local, simulate_model

def model_loss(theta, *args):
    """Return scalar RMSE for a proposed theta.

    args expected: (t_obs, y_obs, param_template, observed_states)
    """
    t_obs, y_obs, param_template, observed_states = args
    res = residuals_for_theta(theta, t_obs, y_obs, param_template, observed_states)
    return float(np.sqrt(np.mean(res ** 2)))

print("Model loss now computes RMSE by integrating the model and comparing observed states.")

# Demo: create synthetic data by simulating default params
p0 = LGParams()
t_obs = np.linspace(0, 48, 40)
y0 = default_initial_conditions()
X = simulate_model(p0, y0, t_obs)
# pick mRNA states only (M_P, M_C, M_B, M_R indices 0, 3, 6, 9)
observed_states = [0, 3, 6, 9]
y_obs = X[observed_states] + 0.02 * np.random.randn(len(observed_states), t_obs.size)

# initial theta (same order as theta_to_params)
theta0 = np.array([p0.v_sP, p0.v_sC, p0.v_sB, p0.v_sR,
                   p0.k_degM_P, p0.k_degM_C, p0.k_degM_B, p0.k_degM_R])

# quick local fit demo
res = fit_local(theta0, None, (t_obs, y_obs), param_template=p0, observed_states=observed_states)
print('Local fit RMSE:', res.rmse)
print('Optimized theta:', res.x)
